In [1]:
import openai
import os
from langchain.llms import OpenAI
openai.api_key = os.getenv("OPENAI_API_KEY")
# print(os.environ)
# print(os.environ["OPENAI_API_KEY"] )
# print(openai.api_key)

In [2]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "大谷翔平について教えて"},
    ],
)
print(response.choices[0]["message"]["content"].strip())

大谷翔平は、日本のプロ野球選手であり、現在はアメリカのロサンゼルス・エンゼルスに所属しています。ポジションは投手と外野手を兼任しており、二刀流として有名です。

1994年7月5日生まれで、滋賀県出身です。高校時代には、甲子園でエースとして活躍し、2012年のドラフト会議で福岡ソフトバンクホークスから指名を受けました。プロ入り後は、主に投手として活躍し、2016年には最多勝を獲得し、リーグ優勝に貢献しました。

2017年には、メジャーリーグベースボール（MLB）のロサンゼルス・エンゼルスに移籍し、外野手としても出場しました。2018年には、MLB史上初となる同時期に20本以上の本塁打を放ち、10勝以上を挙げた二刀流選手として、注目を集めました。

大谷翔平は、抜群の身体能力と技術を持ち合わせ、球界を代表するスター選手の一人として知られています。


In [3]:
# モデル名を指定しない場合、「text-davinci-003」が利用されます
llm = OpenAI(temperature=0.9)
text = "技術記事のタイトルをよしなに生成してくれるAIの名称を考えてください。"
print(llm(text))



『タイトルマジック』


In [10]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["target"],
    template="技術記事のタイトルをよしなに生成してくれるAIの{target}を考えてください。",
)
print(llm(prompt=prompt.format(target="セールスポイント")))



・秒速で最適なテクニカル記事タイトルを自動生成
・広範なトピックをサポート
・最新のテクノロジーに完全対応
・複数のクリエイティブなタイトルを同時生成
・タイトルの品質を保証する機能も搭載


In [12]:
from langchain.chains import LLMChain, SimpleSequentialChain

llm = OpenAI(temperature=.7)
template = """あなたは新商品の開発を担当しています。新商品のターゲットが与えられたら、ターゲットに売れる新商品のアイデアを出してください。

ターゲット: {target}
担当者: 以下が新商品のアイデアです。"""
prompt_template = PromptTemplate(input_variables=["target"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

llm = OpenAI(temperature=.7)
template = """あなたは経営者です。新商品のアイデアが与えられたら、経営者の観点から批判的にレビューをしてください。

アイデア:
{idea}
経営者：
"""
prompt_template = PromptTemplate(input_variables=["idea"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

overall_chain = SimpleSequentialChain(chains=[synopsis_chain, review_chain], verbose=True)
review = overall_chain.run("通勤に1時間以上かかっている20代の男性")



> Entering new SimpleSequentialChain chain...


1. 通勤用のスマートフォンアプリ: 通勤時間を最適化するための道案内、乗り換え検索、交通情報などの機能を備えたスマートフォンアプリ。

2. コミュニケーションツール: 通勤中に他の乗客とコミュニケーションを取るためのツール。チャット機能などを備えている。

3. モバイルオフィス: 通勤中に仕事を行うためのモバイルオフィス。パソコンやプリンターなどオフィスに必要な機器を簡単に持ち運

1. 通勤用のスマートフォンアプリ: このアイデアは、乗客がより効率的な通勤を行えるようになることで、便利なサービスとなるだろう。しかし、データ収集を行う際にプライバシー上の問題があることも考慮する必要がある。また、技術的なハードルがあるかもしれない。

2. コミュニケーションツール: このツールは、通勤中の乗客が楽しい体験を得られるようになるだろう。しかし、無料のアプリとしての提供が可能かどうかや、

> Finished chain.


In [4]:
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI
from langchain import LLMMathChain, SerpAPIWrapper

llm = OpenAI(temperature=0)
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    )
]

llm = OpenAI(temperature=0)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
agent.run("日本で一番高い山の高さは何メートルですか?その高さの5乗根を求めてください。")
# agent.run("韓国の覚えておくべき文化，風習を教えてください")



> Entering new AgentExecutor chain...
 I need to find the height of the highest mountain in Japan and then calculate its fifth root.
Action: Search
Action Input: "highest mountain in Japan"
Observation: Mount Fuji
Thought: I need to find the height of Mount Fuji
Action: Search
Action Input: "height of Mount Fuji"
Observation: 12,388′
Thought: I need to convert this to meters
Action: Calculator
Action Input: 12,388 * 0.3048

> Entering new LLMMathChain chain...
12,388 * 0.3048
```python
print(12388 * 0.3048)
```

Answer: 3775.8624

> Finished chain.

Observation: Answer: 3775.8624

Thought: I now need to calculate the fifth root of this number
Action: Calculator
Action Input: 3775.8624^(1/5)

> Entering new LLMMathChain chain...
3775.8624^(1/5)
```python
print(3775.8624**(1/5))
```

Answer: 5.1928195584457635

> Finished chain.

Observation: Answer: 5.1928195584457635

Thought: I now know the final answer
Final Answer: The height of the highest mountain in Japan is 3775.8624 meters an

'The height of the highest mountain in Japan is 3775.8624 meters and its fifth root is 5.1928195584457635.'